In [1]:
import sys
sys.path.append('C:\\Program Files\\ASAP 2.2\\bin')

import multiresolutionimageinterface as mir
import cv2
from tqdm import tqdm_notebook
import numpy as np
import os
import matplotlib.pyplot as plt


In [7]:
reader = mir.MultiResolutionImageReader()

In [8]:
dirData = '../data/training/'
ImageFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirData):
    for file in f:
        if '.tif' in file and 'mask' not in file:
            ImageFiles.append(os.path.join(r, file))

In [9]:
## This function is adapted from a digital pathology pipeline code of Mikko Tukiainen
# utils 폴더의 wsi2tissueMask.py 파일의 함수들과 동일.
# 배경은 0으로, 조직부분은 255로 저장
def make_tissue_mask(slide, mask_level = 4, morpho=None, morpho_kernel_size=5, morpho_iter=1, median_filter=False, return_original = False): 
    ''' make tissue mask
        return tissue mask array which has tissue locations (pixel value 0 -> empty, 255 -> tissue)
    Args:
        slide (MultiResolutionImage): MultiResolutionImage slide to process
        mask_level (int): defines the level of zoom at which the mask be created (default 4)
        morpho (cv2.MORPHO): OpenCV morpho flag, Cv2.MORPHO_OPEN or Cv2.MORPHO_CLOSE (default None)
        morpho_kernel_size (int): kernel size for morphological transformation (default 5)
        morpho_iter (int): morphological transformation iterations (default=1)
        median_filtern (bool): Use median filtering to remove noise (default False)
        return_original (bool): return also the unmasked image
    '''
    
    # 슬라이드 읽기
    ds = slide.getLevelDownsample(mask_level)
    original_tissue = slide.getUCharPatch(0,
                                         0,
                                         int(slide.getDimensions()[0] / float(ds)),
                                         int(slide.getDimensions()[1] / float(ds)),
                                         mask_level)
    
    # 마스크의 밝기 채널만 추출하여 threshold에 따라 이진화
    tissue_mask = cv2.cvtColor(np.array(original_tissue), cv2.COLOR_RGBA2RGB)
    tissue_mask = cv2.cvtColor(tissue_mask, cv2.COLOR_BGR2HSV)
    tissue_mask = tissue_mask[:, :, 1]
    _, tissue_mask = cv2.threshold(tissue_mask, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    # morphology 변환
    if morpho is not None:
        kernel = np.ones((morpho_kernel_size,morpho_kernel_size), np.uint8)
        tissue_mask = cv2.morphologyEx(tissue_mask, morpho, kernel, iterations = morpho_iter)
    
    # 중앙값 필터링으로 노이즈 제거
    if median_filter:
        tissue_mask = cv2.medianBlur(tissue_mask, 15)
    
    # 마스크를 넘파이 배열로 변환
    tissue_mask = np.array(tissue_mask, dtype=np.uint8)

    # 원본도 같이 저장할지
    if return_original:
        return tissue_mask, original_tissue
    else:
        return tissue_mask

In [10]:
def CreateTissueMask(tifPath):
    
    # 파일 명만 가져오기
    fileNamePart = tifPath.replace('.tif','').replace(dirData, "")
    
    # Skip if this mask is already found
    maskPath = tifPath.replace('.tif', '_tissue_mask_ds16.npy')
    if (os.path.isfile(maskPath)):
        print('Info - Tissue mask file of {0} already exists - skipping'.format(tifPath))
        return
    
    # create tissue mask
    mr_image = reader.open(tifPath)
    if(mr_image is None):
        print('Warning - Could not read {0} - skipping'.format(tifPath))
        return
    tissue_mask = make_tissue_mask(mr_image,
                                   mr_image.getBestLevelForDownSample(16), 
                                   morpho=cv2.MORPH_CLOSE,
                                   morpho_kernel_size=7,
                                   morpho_iter=2,
                                   median_filter=True)
    # tissue_mask is a binary array dtype.uint8  (16 times downsampled)
    np.save(maskPath, tissue_mask)

In [11]:
for f in tqdm_notebook(ImageFiles, 'Creating tissue masks...'):
    print('WSI: ' + f)
    CreateTissueMask(f)

C:\Users\songhow53\AppData\Local\Temp\ipykernel_17720\1165693191.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f in tqdm_notebook(ImageFiles, 'Creating tissue masks...'):


Creating tissue masks...:   0%|          | 0/6 [00:00<?, ?it/s]

WSI: ../data/training/center_0\patient_001_node_1.tif
WSI: ../data/training/center_0\patient_004_node_0.tif
WSI: ../data/training/center_0\patient_004_node_1.tif
WSI: ../data/training/center_0\patient_004_node_2.tif
WSI: ../data/training/center_0\patient_004_node_3.tif
WSI: ../data/training/center_0\patient_004_node_4.tif
